# Comparing channels

- R: sequential
- G: reverse complementary
- B: not match at all

In [1]:
import ppscore
import pandas
import os
import numpy
from scipy.stats import spearmanr
from IPython.core.display import display, HTML
from ete3 import Tree

/tmp/ipykernel_107592/499083497.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
tree_path = "../../data/trees"

In [3]:
def read_and_compare(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_df = pandas.read_csv(f"{path}/Control with Clustal Omega.csv", index_col=0)
    for file in os.listdir(path):
        if file.endswith(".csv"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                df = pandas.read_csv(f"{path}/{file}", index_col=0)
                result_dict[basename] = round(numpy.sqrt(numpy.sum((control_df.values - df.values)**2)), 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Euclidean Distance from distance matrices

In [4]:
channels = ("full", "red", "green", "blue", "red_green", "red_blue", "green_blue", "gray_r", "gray_g", "gray_b", "gray_max", "gray_mean")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin", "orthologs_androglobin"]
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_compare(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,full,Control with Clustal Omega,Local with Smith–Waterman,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,,,,
orthologs_hemoglobin_beta,0.9085,1.1081,0.0,0.8937,1.0098,0.4040,0.3869,0.3705,0.3708,0.3923,1.0630,1.1279,1.1527,1.1396,2.3386
orthologs_myoglobin,0.9196,1.0228,0.0,0.9044,0.9527,0.4211,0.4276,0.3084,0.3213,0.3419,1.0055,1.0761,1.0707,1.1520,2.3185
orthologs_neuroglobin,1.8043,0.7903,0.0,1.1514,5.6899,1.1500,1.1371,3.4429,3.4324,3.4336,5.8481,5.8718,5.8561,1.7182,5.7873
orthologs_cytoglobin,6.3417,1.9134,0.0,3.1095,5.2573,2.0361,1.4018,4.0742,3.4607,5.4473,5.4216,10.5411,7.7587,3.2222,9.5408
orthologs_androglobin,8.2855,11.2361,0.0,2.9601,10.3009,3.4653,3.0137,7.0305,6.8015,7.5320,10.5586,12.5628,11.3761,1.5366,11.9055


In [5]:
def read_and_linear_correlate(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_df = pandas.read_csv(f"{path}/Control with Clustal Omega.csv", index_col=0)
    for file in os.listdir(path):
        if file.endswith(".csv"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                df = pandas.read_csv(f"{path}/{file}", index_col=0)
                result_dict[basename] = round(spearmanr(control_df.values.flatten(), df.values.flatten())[0], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Linear Correlation from distance matrices

In [6]:
channels = ("full", "red", "green", "blue", "red_green", "red_blue", "green_blue", "gray_r", "gray_g", "gray_b", "gray_max", "gray_mean")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin", "orthologs_androglobin"]
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_linear_correlate(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

/home/dmvieira/.virtualenvs/bioinfo2.0/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/dmvieira/.virtualenvs/bioinfo2.0/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/dmvieira/.virtualenvs/bioinfo2.0/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/dmvieira/.virtualenvs/bioinfo2.0/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning

index,Global with Needleman-Wunsch,full,Control with Clustal Omega,Local with Smith–Waterman,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,,,,
orthologs_hemoglobin_beta,0.9999,0.9972,1.0,0.9998,0.9970,0.9972,0.9968,0.9975,0.9973,0.9972,0.9970,0.9972,0.9968,NaN,0.9190
orthologs_myoglobin,0.9999,0.9925,1.0,0.9998,0.9957,0.9963,0.9914,0.9937,0.9948,0.9941,0.9959,0.9963,0.9914,NaN,0.9489
orthologs_neuroglobin,0.9999,0.9972,1.0,0.9992,0.9737,0.9910,0.9773,0.9752,0.9735,0.9915,0.9725,0.9899,0.9769,NaN,0.9377
orthologs_cytoglobin,0.7775,0.5263,1.0,0.8623,0.5072,0.5351,0.5871,0.6130,0.5652,0.5255,0.5074,0.5349,0.5871,NaN,0.6114
orthologs_androglobin,0.9054,0.7830,1.0,0.9769,0.7430,0.3621,0.7820,0.7753,0.7727,0.7359,0.7565,0.3604,0.7820,NaN,0.8153


In [7]:
def read_and_correlate(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_df = pandas.read_csv(f"{path}/Control with Clustal Omega.csv", index_col=0)
    for file in os.listdir(path):
        if file.endswith(".csv"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                df = pandas.read_csv(f"{path}/{file}", index_col=0)
                result_dict[basename] = round(ppscore.score(
                    pandas.DataFrame({"x": control_df.values.flatten(), "y": df.values.flatten()}), "x", "y")["ppscore"], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Predictive Power Score from distance matrices

In [8]:
channels = ("full", "red", "green", "blue", "red_green", "red_blue", "green_blue", "gray_r", "gray_g", "gray_b", "gray_max", "gray_mean")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin", "orthologs_androglobin"]
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_correlate(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,full,Control with Clustal Omega,Local with Smith–Waterman,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,,,,
orthologs_hemoglobin_beta,0.9953,0.9651,0.9968,0.9944,0.9650,0.9629,0.9654,0.9648,0.9673,0.9668,0.9646,0.9629,0.9654,0.0,0.8945
orthologs_myoglobin,0.9954,0.9494,0.9968,0.9925,0.9564,0.9461,0.9341,0.9528,0.9539,0.9394,0.9561,0.9461,0.9341,0.0,0.8670
orthologs_neuroglobin,0.9876,0.9705,0.9964,0.9897,0.9641,0.9654,0.9632,0.9647,0.9638,0.9649,0.9642,0.9656,0.9632,0.0,0.9325
orthologs_cytoglobin,0.8524,0.7834,0.9913,0.8584,0.7194,0.7880,0.7608,0.7630,0.7425,0.7511,0.7194,0.7884,0.7609,0.0,0.7956
orthologs_androglobin,0.9773,0.9363,0.9977,0.9799,0.9452,0.8619,0.9307,0.9386,0.9406,0.9065,0.9452,0.8619,0.9307,0.0,0.8757


In [9]:
def read_and_tree_compare(dataset, channel="full", control="Control with Clustal Omega"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_tree = Tree(f"{path}/{control}.nw", format=1)
    for file in os.listdir(path):
        if file.endswith(".nw"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                tree = Tree(f"{path}/{file}", format=1)
                result = control_tree.compare(tree, unrooted=True)
                result_dict[basename] = round(result["norm_rf"], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Robinson-foulds distance from generated Trees

In [10]:
channels = ("full", "red", "green", "blue", "red_green", "red_blue", "green_blue", "gray_r", "gray_g", "gray_b", "gray_max", "gray_mean")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin", "orthologs_androglobin"]
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,,,,
orthologs_hemoglobin_beta,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0,0.2500
orthologs_myoglobin,0.0000,0.0,0.1667,0.0,0.0833,0.1667,0.0833,0.0833,0.1667,0.1667,0.0833,0.1667,0.0833,1.0,0.1667
orthologs_neuroglobin,0.0000,0.0,0.0000,0.0,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,1.0,0.2500
orthologs_cytoglobin,0.6667,0.5,0.6667,0.0,0.6667,0.8333,0.6667,0.6667,0.5833,0.7500,0.6667,0.8333,0.6667,1.0,0.7500
orthologs_androglobin,0.8333,0.5,0.8333,0.0,0.9167,0.9167,0.8333,0.8333,0.8333,0.8333,0.9167,0.9167,0.8333,1.0,0.7500


In [11]:
channels = ("full", "red", "green", "blue", "red_green", "red_blue", "green_blue", "gray_r", "gray_g", "gray_b", "gray_max", "gray_mean")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin", "orthologs_androglobin"]
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare(dataset, channel, "Global with Needleman-Wunsch"))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,,,,
orthologs_hemoglobin_beta,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0,0.2500
orthologs_myoglobin,0.0,0.0000,0.1667,0.0000,0.0833,0.1667,0.0833,0.0833,0.1667,0.1667,0.0833,0.1667,0.0833,1.0,0.1667
orthologs_neuroglobin,0.0,0.0000,0.0000,0.0000,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,1.0,0.2500
orthologs_cytoglobin,0.0,0.2500,0.5000,0.6667,0.7500,0.6667,0.5833,0.5833,0.6667,0.5833,0.7500,0.6667,0.5833,1.0,0.5833
orthologs_androglobin,0.0,0.4167,0.6667,0.8333,0.8333,0.5833,0.6667,0.5833,0.5833,0.6667,0.8333,0.5833,0.6667,1.0,0.6667


In [12]:
channels = ("full", "red", "green", "blue", "red_green", "red_blue", "green_blue", "gray_r", "gray_g", "gray_b", "gray_max", "gray_mean")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin", "orthologs_androglobin"]
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare(dataset, channel, "Local with Smith–Waterman"))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,,,,
orthologs_hemoglobin_beta,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0,0.2500
orthologs_myoglobin,0.0000,0.0,0.1667,0.0,0.0833,0.1667,0.0833,0.0833,0.1667,0.1667,0.0833,0.1667,0.0833,1.0,0.1667
orthologs_neuroglobin,0.0000,0.0,0.0000,0.0,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,1.0,0.2500
orthologs_cytoglobin,0.2500,0.0,0.5000,0.5,0.5833,0.6667,0.5833,0.5833,0.5000,0.5833,0.5833,0.6667,0.5833,1.0,0.6667
orthologs_androglobin,0.4167,0.0,0.5833,0.5,0.6667,0.6667,0.5833,0.5000,0.5000,0.5833,0.6667,0.6667,0.5833,1.0,0.5833


In [13]:
def read_and_tree_compare_branches(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_tree = Tree(f"{path}/Control with Clustal Omega.nw", format=1)
    for file in os.listdir(path):
        if file.endswith(".nw"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                tree = Tree(f"{path}/{file}", format=1)
                result = control_tree.compare(tree, unrooted=True)
                result_dict[basename] = 1.0 - round(result["source_edges_in_ref"], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Compatibility branch score from generated Trees

In [14]:
channels = ("full", "red", "green", "blue", "red_green", "red_blue", "green_blue", "gray_r", "gray_g", "gray_b", "gray_max", "gray_mean")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin", "orthologs_androglobin"]
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare_branches(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,,,,
orthologs_hemoglobin_beta,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.4615,0.1154
orthologs_myoglobin,0.0000,0.0000,0.0769,0.0,0.0385,0.0769,0.0385,0.0385,0.0769,0.0769,0.0385,0.0769,0.0385,0.4615,0.0769
orthologs_neuroglobin,0.0000,0.0000,0.0000,0.0,0.0385,0.0385,0.0385,0.0385,0.0385,0.0385,0.0385,0.0385,0.0385,0.4615,0.1154
orthologs_cytoglobin,0.3077,0.2308,0.3077,0.0,0.3077,0.3846,0.3077,0.3077,0.2692,0.3462,0.3077,0.3846,0.3077,0.4615,0.3462
orthologs_androglobin,0.3846,0.2308,0.3846,0.0,0.4231,0.4231,0.3846,0.3846,0.3846,0.3846,0.4231,0.4231,0.3846,0.4615,0.3462


In [15]:
def read_and_compare_channels(dataset):
    result_dict = {}
    dfs = {}
    channels = ("full", "red", "green", "blue", "red_green", "red_blue", "green_blue", "gray_r", "gray_g", "gray_b", "gray_max", "gray_mean")
    for channel in channels:
        result_dict[channel] = {}
        path = f"{tree_path}/{channel}/{dataset}"
        dfs[channel] = pandas.read_csv(f"{path}/Structural Similarity Index Measure.csv", index_col=0)
    for channel in channels:
        control_df = dfs[channel]
        for c, df in dfs.items():
            result_dict[channel][c] = round(numpy.sqrt(numpy.sum((control_df.values - df.values)**2)), 4)
    result_df = pandas.DataFrame.from_dict(result_dict, orient='index')
    result_df.index.name = "dataset"
    return result_df.style.background_gradient(axis=None, vmin=result_df.min().min(), vmax=result_df.max().max(), cmap="YlGnBu")

## Features Euclidean distance from distance matrices

In [16]:
for dataset in datasets:
    try:
        correlations = read_and_compare_channels(dataset)
    except FileNotFoundError:
        continue
    else:
        display(HTML(f"<center><h3>{dataset}</h3></center>"))
        display(correlations)        

,full,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,
full,0.000000,0.109300,1.483700,1.473300,0.759700,0.749000,0.725400,0.062000,0.059700,0.093000,2.233800,1.270300
red,0.109300,0.000000,1.382000,1.372200,0.657800,0.648400,0.625300,0.054100,0.131900,0.184700,2.132100,1.362700
green,1.483700,1.382000,0.000000,0.041700,0.724500,0.735800,0.758900,1.435900,1.501100,1.534800,0.750600,2.712000
blue,1.473300,1.372200,0.041700,0.000000,0.715700,0.724600,0.748100,1.426100,1.492000,1.522500,0.761300,2.704800
red_green,0.759700,0.657800,0.724500,0.715700,0.000000,0.046500,0.053200,0.711700,0.777500,0.814500,1.474800,1.998800
red_blue,0.749000,0.648400,0.735800,0.724600,0.046500,0.000000,0.031900,0.702200,0.769100,0.799700,1.485400,1.993100
green_blue,0.725400,0.625300,0.758900,0.748100,0.053200,0.031900,0.000000,0.679100,0.744700,0.776600,1.508800,1.969000
gray_r,0.062000,0.054100,1.435900,1.426100,0.711700,0.702200,0.679100,0.000000,0.086200,0.143600,2.186100,1.311200
gray_g,0.059700,0.131900,1.501100,1.492000,0.777500,0.769100,0.744700,0.086200,0.000000,0.125000,2.251700,1.245800


,full,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,
full,0.000000,0.116300,1.421600,1.424300,0.733900,0.714400,0.696200,0.098700,0.112300,0.085300,2.159900,1.385400
red,0.116300,0.000000,1.356100,1.360400,0.670400,0.651200,0.635300,0.053300,0.159700,0.178900,2.094300,1.417700
green,1.421600,1.356100,0.000000,0.039000,0.688900,0.708200,0.728000,1.409300,1.478800,1.470800,0.739400,2.724700
blue,1.424300,1.360400,0.039000,0.000000,0.692200,0.710800,0.730900,1.413500,1.483500,1.472500,0.736200,2.732400
red_green,0.733900,0.670400,0.688900,0.692200,0.000000,0.041300,0.074300,0.723400,0.795400,0.786500,1.426800,2.058700
red_blue,0.714400,0.651200,0.708200,0.710800,0.041300,0.000000,0.053100,0.704200,0.775400,0.764500,1.446300,2.044100
green_blue,0.696200,0.635300,0.728000,0.730900,0.074300,0.053100,0.000000,0.688000,0.754500,0.744200,1.466400,2.031600
gray_r,0.098700,0.053300,1.409300,1.413500,0.723400,0.704200,0.688000,0.000000,0.124300,0.152400,2.147400,1.368400
gray_g,0.112300,0.159700,1.478800,1.483500,0.795400,0.775400,0.754500,0.124300,0.000000,0.116900,2.218200,1.326000


,full,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,
full,0.000000,4.995600,0.393900,0.386500,2.726300,2.718300,2.721400,5.154600,5.179800,5.167600,2.243400,5.137500
red,4.995600,0.000000,4.738000,4.746400,2.277600,2.285200,2.280300,0.159600,0.188600,0.204900,7.206900,0.861000
green,0.393900,4.738000,0.000000,0.034200,2.460500,2.453400,2.458500,4.897300,4.923700,4.912700,2.468900,4.898600
blue,0.386500,4.746400,0.034200,0.000000,2.469100,2.461400,2.466500,4.905600,4.932000,4.920400,2.460900,4.903100
red_green,2.726300,2.277600,2.460500,2.469100,0.000000,0.043100,0.049200,2.436800,2.463400,2.453700,4.929400,2.516500
red_blue,2.718300,2.285200,2.453400,2.461400,0.043100,0.000000,0.025300,2.444400,2.470800,2.459500,4.922200,2.512600
green_blue,2.721400,2.280300,2.458500,2.466500,0.049200,0.025300,0.000000,2.439500,2.465700,2.454300,4.927300,2.507200
gray_r,5.154600,0.159600,4.897300,4.905600,2.436800,2.444400,2.439500,0.000000,0.043100,0.108700,7.366200,0.858200
gray_g,5.179800,0.188600,4.923700,4.932000,2.463400,2.470800,2.465700,0.043100,0.000000,0.098200,7.392600,0.855300


,full,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,
full,0.000000,4.116300,0.511800,0.840100,2.565800,2.057900,3.939800,4.275100,9.093500,6.296400,4.332700,8.085100
red,4.116300,0.000000,4.339100,4.627000,2.285800,2.119800,3.358100,0.172300,7.248000,3.987200,7.855200,6.159500
green,0.511800,4.339100,0.000000,1.096800,2.583100,2.232600,3.796500,4.491500,8.922700,6.232300,4.463100,7.944300
blue,0.840100,4.627000,1.096800,0.000000,3.309400,2.680200,4.737800,4.793600,9.900800,7.074200,3.537400,8.888600
red_green,2.565800,2.285800,2.583100,3.309400,0.000000,0.962600,1.798000,2.384500,6.774000,3.786600,6.840800,5.720400
red_blue,2.057900,2.119800,2.232600,2.680200,0.962600,0.000000,2.711100,2.267500,7.682100,4.607700,6.139900,6.614400
green_blue,3.939800,3.358100,3.796500,4.737800,1.798000,2.711100,0.000000,3.376900,5.225600,2.647100,8.228300,4.267700
gray_r,4.275100,0.172300,4.491500,4.793600,2.384500,2.267500,3.376900,0.000000,7.149100,3.891500,8.027200,6.063100
gray_g,9.093500,7.248000,8.922700,9.900800,6.774000,7.682100,5.225600,7.149100,0.000000,3.286600,13.385700,1.296600


,full,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,
full,0.000000,3.153500,8.076600,8.311100,4.220300,4.562700,3.939700,3.118700,2.871000,0.355200,12.529700,2.136400
red,3.153500,0.000000,7.733200,7.639400,4.045100,3.776400,4.618700,0.258500,6.018600,3.500800,11.612900,5.239400
green,8.076600,7.733200,0.000000,0.842200,3.937000,3.966600,4.211800,7.982700,9.184100,8.175600,4.592300,8.557300
blue,8.311100,7.639400,0.842200,0.000000,4.096900,3.956000,4.578900,7.894700,9.644800,8.441600,4.220800,8.975600
red_green,4.220300,4.045100,3.937000,4.096900,0.000000,0.815200,1.156300,4.271900,5.905400,4.369000,8.311300,5.152400
red_blue,4.562700,3.776400,3.966600,3.956000,0.815200,0.000000,1.968500,4.022700,6.522600,4.759500,8.099000,5.737800
green_blue,3.939700,4.618700,4.211800,4.578900,1.156300,1.968500,0.000000,4.811400,5.094200,3.999500,8.769100,4.410100
gray_r,3.118700,0.258500,7.982700,7.894700,4.271900,4.022700,4.811400,0.000000,5.987200,3.465900,11.871300,5.221400
gray_g,2.871000,6.018600,9.184100,9.644800,5.905400,6.522600,5.094200,5.987200,0.000000,2.526500,13.776300,0.925700
